In [16]:
import pandas as pd
import os
import numpy as np
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from PIL import Image
from sklearn.model_selection import train_test_split

# Constants
TARGET_SIZE = (180, 180)


In [ ]:
def create_data(root_dir):

    # Lists to store data
    all_images = []
    y = []
    full_path_all = []

    # Define a mapping from folder name to label
    disease_to_label = {
        "Eczema": 0,
        "Melanoma": 1,
        "Atopic_Dermatitis": 2,
        "Basal_Cell_Carcinoma": 3,
        "Melanocytic_Nevi": 4,
        "Benign_Keratosis_like_Lesions": 5,
        "Psoriasis": 6,
        "Seborrheic_Keratoses": 7,
        "Tinea": 8,
        "Warts": 9
    }

    # Function to process a directory
    def process_directory(directory):
        for item in os.listdir(directory):
            full_path = os.path.join(directory, item)

            # If it's a directory, process it recursively
            if os.path.isdir(full_path):
                process_directory(full_path)

            # If it's an image file, add it to our lists
            elif item.lower().endswith(('.png', '.jpg', '.jpeg')):
                all_images.append(item)

                # Assign a label based on the folder name
                for disease, label in disease_to_label.items():
                    if disease in directory:
                        y.append(label)
                        break

                # Add full path of the image to the list
                full_path_all.append(full_path)

    # Start processing from root directory
    process_directory(root_dir)

    # Create DataFrame
    df = pd.DataFrame({
        'image': all_images,
        'y': y,
        'full_path': full_path_all
    })

    return df


In [18]:
# Image filtering functions
def resize_image(image_array):
    """Resize image to target size"""
    return cv2.resize(image_array, TARGET_SIZE)

def gaussian_blur_image(image_array):
    """Apply Gaussian blur to reduce noise"""
    return cv2.GaussianBlur(image_array, (5, 5), 0)

def denoise_image(image_array):
    """Apply fast non-local means denoising"""
    return cv2.fastNlMeansDenoisingColored(image_array, None, 0.1, 0.1, 7, 21)

def sharpen_image(image_array):
    """Sharpen image using a custom kernel"""
    sharpening_kernel = np.array([[0, -1, 0],
                                   [-1, 5, -1],
                                   [0, -1, 0]])
    return cv2.filter2D(image_array, -2, sharpening_kernel)

In [19]:
# Image Augmentation functions
def horizontal_flip_image(image_array):
    """Flip image horizontally"""
    h_flipped_img = cv2.flip(image_array, 1)
    return cv2.resize(h_flipped_img, TARGET_SIZE)

def vertical_flip_image(image_array):
    """Flip image vertically"""
    v_flipped_img = cv2.flip(image_array, 0)
    return cv2.resize(v_flipped_img, TARGET_SIZE)

def rotate_image(image_array, degree=cv2.ROTATE_90_CLOCKWISE):
    """Rotate image 90 degrees"""
    rotated = cv2.rotate(image_array, degree)
    return cv2.resize(rotated, TARGET_SIZE)

def crop_image(image_array, crop_center_percent=0.6):
    """Crop center portion of the image"""
    h, w, c = image_array.shape
    crop_size = int(min(h, w) * crop_center_percent)  
    start_x = (w - crop_size) // 2
    start_y = (h - crop_size) // 2
    cropped = image_array[start_y:start_y + crop_size, start_x:start_x + crop_size]
    return cv2.resize(cropped, TARGET_SIZE)

In [20]:
def filter_and_augment_image(image_path):
    """
    Apply filtering and augmentation to an image
    
    Args:
        image_path (str): Path to the image file
    
    Returns:
        list: List of augmented images
    """
    # Open image and convert to numpy array
    image = cv2.imread(image_path)
    
    # Resizing image
    image = resize_image(image)

    # Apply filtering techniques
    image = denoise_image(image)
    image = gaussian_blur_image(image)
    image = sharpen_image(image)

    # Perform augmentations
    augmented_images = [
        horizontal_flip_image(image),
        vertical_flip_image(image),
        rotate_image(image),
        crop_image(image)
    ]

    return augmented_images

In [21]:
def augment_dataset(df, output_dir='augmented_images'):
    """
    Augment the entire dataset by generating additional images
    
    Args:
        df (pandas.DataFrame): Original dataset
        output_dir (str, optional): Directory to save augmented images
    
    Returns:
        pandas.DataFrame: Augmented dataset
    """
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Create lists to store augmented data
    augmented_images = []
    augmented_labels = []
    augmented_full_paths = []

    # Iterate through original dataset and apply augmentations
    for index, row in df.iterrows():
        # Add original image first
        augmented_images.append(row['image'])
        augmented_labels.append(row['y'])
        augmented_full_paths.append(row['full_path'])

        # Generate augmented images
        aug_imgs = filter_and_augment_image(row['full_path'])
        
        # Add augmented images
        for aug_img_idx, aug_img in enumerate(aug_imgs):
            # Create unique filename for augmented image
            aug_img_filename = f"augmented_{os.path.splitext(row['image'])[0]}_{aug_img_idx}.jpg"
            aug_img_path = os.path.join(output_dir, aug_img_filename)
            
            # Save augmented image (will overwrite if exists)
            cv2.imwrite(aug_img_path, aug_img)
            
            # Add to lists
            augmented_images.append(aug_img_filename)
            augmented_labels.append(row['y'])
            augmented_full_paths.append(aug_img_path)

    # Create new augmented DataFrame
    augmented_df = pd.DataFrame({
        'image': augmented_images,
        'y': augmented_labels,
        'full_path': augmented_full_paths
    })

    return augmented_df

In [22]:
# CNN Model Architecture
class SkinDiseaseClassifier(nn.Module):
    def __init__(self, num_classes=10):  
        super(SkinDiseaseClassifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 45 * 45, 128)
        self.fc2 = nn.Linear(128, num_classes) 
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        # The input is an image of size 3×180×180 (channels, width, height).
        x = self.conv1(x)
        # Output: 32×180×180 (3 -> 32 since 32 output channels; 180 is unchanged due to padding)
        x = self.relu(x)
        x = self.pool(x)
        # Output: 32x90x90 (since maxpooling of 2x2 with stride 2 reduces heigh x width by 2)
        x = self.conv2(x)
        # Output: 64×180×180 (32 -> 64 since 64 output channels; 180 is unchanged due to padding)
        x = self.relu(x)
        x = self.pool(x)
        # Output: 64x45x45 (since maxpooling of 2x2 with stride 2 reduces heigh x width by 2)
        x = x.view(-1, 64 * 45 * 45)
        # Flattened the image to a 1D array where 64*45*45 is the number of pixels (64- channels, 45 is width and height)
        # Output: 1 x 129600
        x = self.fc1(x)
        # Takes the 129600 input and maps to 128 vectors (fully connected layers, given as per in the __init__) 
        # Output: 1 x 128
        x = self.relu(x)
        x = self.dropout(x)
        # Randomly sets 50% of the weights in the vectors, to 0 to prevent overfitting and make model more generalized by forcing other 50% of perceptrons to learn (as in __init__)
        x = self.fc2(x)
        # Final fully connected layer; where 128 vectors are fully connected to just 10 perceptrons (the number of classes)
        return x


In [23]:
# Custom Dataset Class
class SkinDiseaseDataset(Dataset):
    def __init__(self, df, transform=None):
        self.image_paths = df['full_path'].tolist()
        self.labels = df['y'].tolist()
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index):
        image = Image.open(self.image_paths[index]).convert("RGB")
        label = self.labels[index]

        if self.transform:
            image = self.transform(image)

        return image, label


In [28]:
# Set random seed for reproducibility
torch.manual_seed(12)
np.random.seed(12)

# Load dataset
#dataset_path = r'C:\Users\Nkay\Desktop\College\Semester 2\Deep Learning with Pytorch\Project\Dataset'
dataset_path = r'D:\College\Deep learning with pytorch\Dataset'
data = create_data(dataset_path)

# Augment dataset
#augmented_data = augment_dataset(data)

# Split the augmented dataset
train_df, val_test_df = train_test_split(data, test_size=0.4, random_state=12)
val_df, test_df = train_test_split(val_test_df, test_size=0.5, random_state=12)

# Print dataset sizes
print(f"Training set size: {len(train_df)}")
print(f"Validation set size: {len(val_df)}")
print(f"Test set size: {len(test_df)}")

Training set size: 10986
Validation set size: 3662
Test set size: 3663


In [29]:
data

,image,y,full_path
0,0_1.jpg,2,D:\College\Deep learning with pytorch\Dataset\...
1,0_16.jpg,2,D:\College\Deep learning with pytorch\Dataset\...
2,0_17.jpg,2,D:\College\Deep learning with pytorch\Dataset\...
3,0_18.jpg,2,D:\College\Deep learning with pytorch\Dataset\...
4,0_19.jpg,2,D:\College\Deep learning with pytorch\Dataset\...
...,...,...,...
18306,v-warts-plantar-108.jpg,9,D:\College\Deep learning with pytorch\Dataset\...
18307,v-warts-plantar-23.jpg,9,D:\College\Deep learning with pytorch\Dataset\...
18308,v-warts-plantar-27.jpg,9,D:\College\Deep learning with pytorch\Dataset\...
18309,v-warts-plantar-67.jpg,9,D:\College\Deep learning with pytorch\Dataset\...


In [33]:
# Define transforms
transform = transforms.Compose([
    transforms.Resize((180, 180)),
    transforms.ToTensor()
    #transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Create datasets
train_dataset = SkinDiseaseDataset(train_df, transform=transform)
val_dataset = SkinDiseaseDataset(val_df, transform=transform)
test_dataset = SkinDiseaseDataset(test_df, transform=transform)

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


In [34]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [35]:
# Initialize model
num_classes = 10  

model = SkinDiseaseClassifier(num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Define scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

# Training loop
num_epochs = 3
for epoch in range(num_epochs):
    print(f'\n--- Epoch {epoch + 1}/{num_epochs} ---')
    
    # Training phase
    model.train()
    train_loss, train_correct, train_total = 0.0, 0, 0
    print('Training...')
    for batch_idx, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()

        # Print status for every 10 batches
        if (batch_idx + 1) % 10 == 0:
            print(f'Batch {batch_idx + 1}/{len(train_loader)}, Loss: {loss.item():.4f}')
    
    train_accuracy = 100 * train_correct / train_total
    print(f'Train Loss: {train_loss / len(train_loader):.4f}, Train Accuracy: {train_accuracy:.2f}%')

    # Validation phase
    model.eval()
    val_loss, val_correct, val_total = 0.0, 0, 0
    print('Validating...')
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_accuracy = 100 * val_correct / val_total
    print(f'Val Loss: {val_loss / len(val_loader):.4f}, Val Accuracy: {val_accuracy:.2f}%')

    # Step the scheduler
    scheduler.step()

# Test phase
print('\nTesting...')
test_loss, test_correct, test_total = 0.0, 0, 0
model.eval()

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        
        _, predicted = torch.max(outputs.data, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

test_accuracy = 100 * test_correct / test_total
print(f'Test Loss: {test_loss / len(test_loader):.4f}, Test Accuracy: {test_accuracy:.2f}%')



--- Epoch 1/3 ---
Training...
Batch 10/172, Loss: 1.7634
Batch 20/172, Loss: 1.3466
Batch 30/172, Loss: 1.5297
Batch 40/172, Loss: 1.3842
Batch 50/172, Loss: 1.2188
Batch 60/172, Loss: 1.3710
Batch 70/172, Loss: 1.1325
Batch 80/172, Loss: 1.1844
Batch 90/172, Loss: 1.0562
Batch 100/172, Loss: 1.1491
Batch 110/172, Loss: 1.1359
Batch 120/172, Loss: 1.1750
Batch 130/172, Loss: 1.1943
Batch 140/172, Loss: 0.8405
Batch 150/172, Loss: 1.1481
Batch 160/172, Loss: 0.9458
Batch 170/172, Loss: 1.2218
Train Loss: 1.2937, Train Accuracy: 54.85%
Validating...
Val Loss: 1.0317, Val Accuracy: 65.18%

--- Epoch 2/3 ---
Training...
Batch 10/172, Loss: 0.9890
Batch 20/172, Loss: 0.7198
Batch 30/172, Loss: 0.8309
Batch 40/172, Loss: 0.7931
Batch 50/172, Loss: 0.9461
Batch 60/172, Loss: 0.9665
Batch 70/172, Loss: 0.8319
Batch 80/172, Loss: 0.9650
Batch 90/172, Loss: 0.8318
Batch 100/172, Loss: 0.8864
Batch 110/172, Loss: 1.1765
Batch 120/172, Loss: 1.0596
Batch 130/172, Loss: 0.8944
Batch 140/172, Loss:

In [36]:
### Using the pre-trained Resnet50 model

import torchvision.models as models

# Load pre-trained ResNet50 model
model = models.resnet50(weights=True)

# Freeze all layers for now
for param in model.parameters():
    param.requires_grad = False  

# Replace the last fully connected layer with num_ftrs to 10 (10 classes) This is the only tunable layer now.
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)  

# Move the model to the appropriate device if exists
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Define learning rate scheduler
#scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    print(f'\n--- Epoch {epoch + 1}/{num_epochs} ---')
    
    # Training phase
    model.train()
    train_loss, train_correct, train_total = 0.0, 0, 0
    print('Training...')
    for batch_idx, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()

        # Print status for every 10 batches
        if (batch_idx + 1) % 10 == 0:
            print(f'Batch {batch_idx + 1}/{len(train_loader)}, Loss: {loss.item():.4f}')
    
    train_accuracy = 100 * train_correct / train_total
    print(f'Train Loss: {train_loss / len(train_loader):.4f}, Train Accuracy: {train_accuracy:.2f}%')

    # Validation phase
    model.eval()
    val_loss, val_correct, val_total = 0.0, 0, 0
    print('Validating...')
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_accuracy = 100 * val_correct / val_total
    print(f'Val Loss: {val_loss / len(val_loader):.4f}, Val Accuracy: {val_accuracy:.2f}%')

    # Step the scheduler
    scheduler.step()

# Test phase
print('\nTesting...')
test_loss, test_correct, test_total = 0.0, 0, 0
model.eval()

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        
        _, predicted = torch.max(outputs.data, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

test_accuracy = 100 * test_correct / test_total
print(f'Test Loss: {test_loss / len(test_loader):.4f}, Test Accuracy: {test_accuracy:.2f}%')

c:\Users\N30N\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)



--- Epoch 1/5 ---
Training...
Batch 10/172, Loss: 1.3756
Batch 20/172, Loss: 1.2088
Batch 30/172, Loss: 0.9752
Batch 40/172, Loss: 1.2553
Batch 50/172, Loss: 0.8946
Batch 60/172, Loss: 1.0631
Batch 70/172, Loss: 0.9039
Batch 80/172, Loss: 0.7697
Batch 90/172, Loss: 0.6641
Batch 100/172, Loss: 0.9014
Batch 110/172, Loss: 0.7229
Batch 120/172, Loss: 0.6226
Batch 130/172, Loss: 0.7271
Batch 140/172, Loss: 0.7599
Batch 150/172, Loss: 0.8027
Batch 160/172, Loss: 0.7742
Batch 170/172, Loss: 0.7476
Train Loss: 0.8988, Train Accuracy: 69.87%
Validating...
Val Loss: 0.6533, Val Accuracy: 77.55%

--- Epoch 2/5 ---
Training...
Batch 10/172, Loss: 0.6000
Batch 20/172, Loss: 0.6994
Batch 30/172, Loss: 0.5271
Batch 40/172, Loss: 0.5140
Batch 50/172, Loss: 0.4972
Batch 60/172, Loss: 0.7823
Batch 70/172, Loss: 0.7591
Batch 80/172, Loss: 0.6563
Batch 90/172, Loss: 0.7073
Batch 100/172, Loss: 0.6455
Batch 110/172, Loss: 0.5772
Batch 120/172, Loss: 0.4131
Batch 130/172, Loss: 0.6765
Batch 140/172, Loss: